FINAL PROJECT CAPSTONE

I chose to confront myself with the following problem:

In a city of your choice, if someone is looking to open a restaurant, where would you recommend that they open it? Similarly, if a contractor is trying to start their own business, where would you recommend that they setup their office.

When a person starts a business, he needs visibility. That is, he needs his business to be seen by others, hence the importance of advertising. However, advertising is expensive and not everyone can afford it. Therefore, you must be in strategic locations in order to be seen by as many people as possible. In the case of a restaurant, it is therefore necessary to study which avenue or street is the most crowded during lunch and dinner hours. For a business such as an agency, it will therefore be necessary to check throughout the day, at strategic times. You will then have to make a list of the most popular avenues at these times. Of course, you cannot always open your business in the most popular street, which is the one you want because of two main criteria:

1- The most popular streets are often the most expensive ones

2- You have to find a rental on that street and affordable too.

So, making a list will be perfect for someone who wants to start a business. This will allow them to search for a location, starting with the most popular street. Data Description : For this project, I will focus on Paris, the city where I live. In order to address this issue, I will simply make a list of the most popular places in the city of Paris. We will then be able to look for a location around its streets, which are very crowded.


For this project, I will focus on Paris, the city where I live. In order to address this issue, I will simply make a list of the most popular places in the city of Paris. We will then be able to look for a location around its streets, which are very crowded.

Let's start by importing some libraries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
import requests

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          97 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0



geopy-1.22.0         | 63 KB     | ##################################### | 100% 
geographiclib-1.50   | 34 KB     | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: 

Let's define a function that can extract the category of a venue, it will be useful later on.

In [2]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

To explore the venues in Paris, we need our Foursquare ID and secret. 

In [3]:
CLIENT_ID = 'W5TWHZXTBRJ4SGEECYSONRS4KCXCOAOMQHEDWYMWSLPQ1EWO' # Foursquare ID
CLIENT_SECRET = '4LIMGPMWXTMFOOUHGURSCAMHPIAYJHPUHTOS5WTI0JMRLVCE' # Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: W5TWHZXTBRJ4SGEECYSONRS4KCXCOAOMQHEDWYMWSLPQ1EWO
CLIENT_SECRET:4LIMGPMWXTMFOOUHGURSCAMHPIAYJHPUHTOS5WTI0JMRLVCE


Now, I am getting the latitude and longitude of the city of Paris.

In [4]:
address = 'Paris'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

48.8566969 2.3514616


Using this url will give me the popular spots in Paris, I chose the center of Paris "Hôtel de Ville" as reference.
And I chose to get the popular spots in a radius of 2Km (can be modified).

In [5]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&radius={}&v={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, 20000, VERSION)

# send GET request and get trending venues
results = requests.get(url).json()
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '4adcda09f964a520e83321e3',
  'name': 'Cathédrale Notre-Dame de Paris',
  'location': {'address': '6 place du parvis Notre-Dame',
   'crossStreet': 'Place Jean-Paul II',
   'lat': 48.85312443201169,
   'lng': 2.3495614528656006,
   'labeledLatLngs': [{'label': 'display',
     'lat': 48.85312443201169,
     'lng': 2.3495614528656006}],
   'distance': 421,
   'postalCode': '75004',
   'cc': 'FR',
   'city': 'Paris',
   'state': 'Île-de-France',
   'country': 'France',
   'formattedAddress': ['6 place du parvis Notre-Dame (Place Jean-Paul II)',
    '75004 Paris',
    'France']},
  'categories': [{'id': '4bf58dd8d48988d132941735',
    'name': 'Church',
    'pluralName': 'Churches',
    'shortName': 'Church',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/religious_church_',
     'suffix': '.png'},
    'prim

Let's now reshape the data into a dataframe. 

In [6]:
dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """Entry point for launching an IPython kernel.


,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,neighborhood,id
0,Cathédrale Notre-Dame de Paris,Church,6 place du parvis Notre-Dame,Place Jean-Paul II,48.853124,2.349561,"[{'label': 'display', 'lat': 48.85312443201169...",421,75004,FR,Paris,Île-de-France,France,[6 place du parvis Notre-Dame (Place Jean-Paul...,NaN,4adcda09f964a520e83321e3
1,Shakespeare & Company,Bookstore,37 rue de la Bûcherie,NaN,48.852568,2.347096,"[{'label': 'display', 'lat': 48.85256839809226...",559,75005,FR,Paris,Île-de-France,France,"[37 rue de la Bûcherie, 75005 Paris, France]",NaN,4adcda21f964a520f23921e3
2,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,Place de l'Hôtel de Ville,NaN,48.856925,2.351412,NaN,25,75004,FR,Paris,Île-de-France,France,"[Place de l'Hôtel de Ville, 75004 Paris, France]",NaN,4bf41231e5eba59334341f90
3,La Maison d'Isabelle,Bakery,47 ter boulevard Saint-Germain,NaN,48.850007,2.348443,"[{'label': 'display', 'lat': 48.85000666698438...",776,75005,FR,Paris,Île-de-France,France,"[47 ter boulevard Saint-Germain, 75005 Paris, ...",NaN,5b394cae92e7a9002c724cf9
4,Centre Pompidou – Musée National d'Art Moderne,Art Museum,Place Georges Pompidou,Rue Beaubourg,48.860730,2.351660,"[{'label': 'display', 'lat': 48.86072970727395...",449,75004,FR,Paris,Île-de-France,France,"[Place Georges Pompidou (Rue Beaubourg), 75004...",NaN,4adcda0af964a520623421e3


I sort the data and drop the columns that are not meaningful.

In [ ]:
dataframe_filtered.drop(['crossStreet', 'labeledLatLngs', 'distance', 'postalCode', 'cc', 'city', 'state', 'country', 'formattedAddress', 'neighborhood', 'id'], axis = 1, inplace = True)

In [118]:
dataframe_filtered.head()

,name,categories,address,lat,lng
0,Cathédrale Notre-Dame de Paris,Church,6 place du parvis Notre-Dame,48.853124,2.349561
1,Shakespeare & Company,Bookstore,37 rue de la Bûcherie,48.852568,2.347096
2,Place de l'Hôtel de Ville – Esplanade de la Li...,Plaza,Place de l'Hôtel de Ville,48.856925,2.351412
3,La Maison d'Isabelle,Bakery,47 ter boulevard Saint-Germain,48.850007,2.348443
4,Centre Pompidou – Musée National d'Art Moderne,Art Museum,Place Georges Pompidou,48.860730,2.351660


Let's now visualize these datas on a map.

In [119]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

print('Folium installed and imported!')

Solving environment: done

# All requested packages already installed.

Folium installed and imported!


I create a map centered around Paris

In [121]:
# define the map centered around Paris
paris_map = folium.Map(location=[latitude, longitude], zoom_start=13)

# display paris map
paris_map

I add markers that indicate the location and pop-up that indicate the category of the popular spot. 

In [122]:
incidents = folium.map.FeatureGroup()

# loop through the 100 crimes and add each to the incidents feature group
for lat, lng, in zip(dataframe_filtered.lat, dataframe_filtered.lng):
    incidents.add_child(
        folium.features.CircleMarker(
            [lat, lng],
            radius=5, # define how big you want the circle markers to be
            color='yellow',
            fill=True,
            fill_color='blue',
            fill_opacity=0.6
        )
    )

# add pop-up text to each marker on the map
latitudes = list(dataframe_filtered.lat)
longitudes = list(dataframe_filtered.lng)
labels = list(dataframe_filtered.categories)

for lat, lng, label in zip(latitudes, longitudes, labels):
    folium.Marker([lat, lng], popup=label).add_to(paris_map)    
    
# add incidents to map
paris_map.add_child(incidents)

Now we have a dataframe with the most popular spot in a radius of 2Km of the center of Paris and a map that help us to visualize where these popular spots are located.
As we want to help customers to know where they can open a new business or restaurant, they can easily get the information of the most popular spots in the dataframe but also have some idea of where it is located in Paris thanks to the map. 

Indeed, opening a new business/restaurant near a popular spot will give the owner some visibility.